In [1]:
!pip install wandb
!pip install wordcloud
!pip install colour

     |████████████████████████████████| 1.8 MB 11.3 MB/s 
     |████████████████████████████████| 145 kB 11.0 MB/s 
     |████████████████████████████████| 181 kB 52.4 MB/s 
     |████████████████████████████████| 63 kB 1.8 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=bc8d9ce6313b3b7853c9ab6acead55bcf303adda21c795bb4f2f38fc5f3b4873
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [2]:
## Installing font for Hindi for matplotlib ##
!apt-get install -y fonts-lohit-deva
!fc-list :lang=hi family

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
The following NEW packages will be installed:
  fonts-lohit-deva
0 upgraded, 1 newly installed, 0 to remove and 42 not upgraded.
Need to get 78.2 kB of archives.
After this operation, 196 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 fonts-lohit-deva all 2.95.4-2 [78.2 kB]
Fetched 78.2 kB in 1s (84.3 kB/s)
Selecting previously unselected package fonts-lohit-deva.
(Reading database ... 155203 files and directories currently installed.)
Preparing to unpack .../fonts-lohit-deva_2.95.4-2_all.deb ...
Unpacking fonts-lohit-deva (2.95.4-2) ...
Setting up fonts-lohit-deva (2.95.4-2) ...
Processing triggers for fontconfig (2.12.6-0ubuntu2) ...
Lohit Devanagari


In [3]:
import os
import random
import time
import wandb
import re, string
import numpy as np
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from wordcloud import WordCloud, STOPWORDS
from collections import Counter
from colour import Color
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

import tensorflow as tf
from tensorflow.keras import layers
import tensorflow.keras.backend as K
from tensorflow.keras.preprocessing.text import Tokenizer

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#Data Loading

In [5]:
## Download the dataset ##
import requests
import tarfile

def download_data(save_path):

    data_url = r"https://storage.googleapis.com/gresearch/dakshina/dakshina_dataset_v1.0.tar"

    r = requests.get(data_url, allow_redirects=True)
    tar_path = "data_assignment3.tar"

    if r.status_code == 200:
        with open(tar_path, 'wb') as f:
            f.write(r.content)

    tar_file = tarfile.open(tar_path)
    tar_file.extractall(save_path)
    tar_file.close()



# uncomment the line below if downloading data for the 1st time
# download_data("/content/drive/MyDrive/DakshinaDataset")

#Prepocessing the Data

In [6]:
def get_data_files(language):
    """ Function fo read data 
    """

    template = "/content/drive/MyDrive/DakshinaDataset/dakshina_dataset_v1.0/{}/lexicons/{}.translit.sampled.{}.tsv"

    train_tsv = template.format(language, language, "train")
    val_tsv = template.format(language, language, "dev")
    test_tsv = template.format(language, language, "test")

    return train_tsv, val_tsv, test_tsv


def add_start_end_tokens(df, cols, sos="\t", eos="\n"):
    """ Adds EOS and SOS tokens to data 
    """
    def add_tokens(s):  
        # \t = starting token
        # \n = ending token
        return sos + str(s) + eos

    for col in cols:
        df[col] = df[col].apply(add_tokens) 
    
def tokenize(lang, tokenizer=None):
    """ Uses tf.keras tokenizer to tokenize the data/words into characters
    """

    if tokenizer is None:
        tokenizer = Tokenizer(char_level=True)
        tokenizer.fit_on_texts(lang)

        lang_tensor = tokenizer.texts_to_sequences(lang)
        lang_tensor = tf.keras.preprocessing.sequence.pad_sequences(lang_tensor,
                                                            padding='post')

    else: 
        lang_tensor = tokenizer.texts_to_sequences(lang)
        lang_tensor = tf.keras.preprocessing.sequence.pad_sequences(lang_tensor,
                                                            padding='post')

    return lang_tensor, tokenizer

def preprocess_data(fpath, input_lang_tokenizer=None, targ_lang_tokenizer=None):
    """ Reads, tokenizes and adds SOS/EOS tokens to data based on above functions
    """

    df = pd.read_csv(fpath, sep="\t", header=None)

    # Addition of start and end tokens
    add_start_end_tokens(df, [0,1])
    
    input_lang_tensor, input_tokenizer = tokenize(df[1].astype(str).tolist(), 
                                                    tokenizer=input_lang_tokenizer)
    
    targ_lang_tensor, targ_tokenizer = tokenize(df[0].astype(str).tolist(),
                                                    tokenizer=targ_lang_tokenizer) 
    
    dataset = tf.data.Dataset.from_tensor_slices((input_lang_tensor, targ_lang_tensor))
    dataset = dataset.shuffle(len(dataset))
    
    return dataset, input_tokenizer, targ_tokenizer

#Building the Model

In [7]:
def get_layer(name, units, dropout, return_state=False, return_sequences=False):

    if name=="rnn":
        return layers.SimpleRNN(units=units, dropout=dropout, 
                                return_state=return_state,
                                return_sequences=return_sequences)

    if name=="gru":
        return layers.GRU(units=units, dropout=dropout, 
                          return_state=return_state,
                          return_sequences=return_sequences)

    if name=="lstm":
        return layers.LSTM(units=units, dropout=dropout, 
                           return_state=return_state,
                           return_sequences=return_sequences)

#Attention
class BahdanauAttention(tf.keras.layers.Layer):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 = tf.keras.layers.Dense(units)
    self.W2 = tf.keras.layers.Dense(units)
    self.V = tf.keras.layers.Dense(1)

  def call(self, enc_state, enc_out):
    
    enc_state = tf.concat(enc_state, 1)
    enc_state = tf.expand_dims(enc_state, 1)

    score = self.V(tf.nn.tanh(self.W1(enc_state) + self.W2(enc_out)))

    attention_weights = tf.nn.softmax(score, axis=1)

    context_vector = attention_weights * enc_out
    context_vector = tf.reduce_sum(context_vector, axis=1)

    return context_vector, attention_weights


class Encoder(tf.keras.Model):
    def __init__(self, layer_type, n_layers, units, encoder_vocab_size, embedding_dim, dropout):
        super(Encoder, self).__init__()
        self.layer_type = layer_type
        self.n_layers = n_layers
        self.units = units
        self.dropout = dropout
        self.embedding = tf.keras.layers.Embedding(encoder_vocab_size, embedding_dim)
        self.create_rnn_layers()

    def call(self, x, hidden):
        x = self.embedding(x)
        x = self.rnn_layers[0](x, initial_state=hidden)

        for layer in self.rnn_layers[1:]:
            x = layer(x)

        output, state = x[0], x[1:]

        return output, state
    
    def create_rnn_layers(self):
        self.rnn_layers = []

        for i in range(self.n_layers):
            self.rnn_layers.append(get_layer(self.layer_type, self.units, self.dropout,
                                                return_sequences=True,
                                                return_state=True))


    def initialize_hidden_state(self, batch_size):

        if self.layer_type != "lstm":
            return [tf.zeros((batch_size, self.units))]
        else:
            return [tf.zeros((batch_size, self.units))]*2

class Decoder(tf.keras.Model):
    def __init__(self, layer_type, n_layers, units, decoder_vocab_size, embedding_dim, dropout, attention=False):
        super(Decoder, self).__init__()

        self.layer_type = layer_type
        self.n_layers = n_layers
        self.units = units
        self.dropout = dropout
        self.attention = attention
        self.embedding_layer = layers.Embedding(input_dim=decoder_vocab_size, 
                                                output_dim=embedding_dim)
        
        self.dense = layers.Dense(decoder_vocab_size, activation="softmax")
        self.flatten = layers.Flatten()
        if self.attention:
            self.attention_layer = BahdanauAttention(self.units)
        self.create_rnn_layers()

    def call(self, x, hidden, enc_out=None):
        
        x = self.embedding_layer(x)

        if self.attention:
            context_vector, attention_weights = self.attention_layer(hidden, enc_out)
            x = tf.concat([tf.expand_dims(context_vector, 1), x], -1)
        else:
            attention_weights = None

        x = self.rnn_layers[0](x, initial_state=hidden)

        for layer in self.rnn_layers[1:]:
            x = layer(x)

        output, state = x[0], x[1:]

        output = self.dense(self.flatten(output))
        
        return output, state, attention_weights

    def create_rnn_layers(self):
        self.rnn_layers = []    

        for i in range(self.n_layers - 1):
            self.rnn_layers.append(get_layer(self.layer_type, self.units, self.dropout,
                                                return_sequences=True,
                                                return_state=True))
        
        self.rnn_layers.append(get_layer(self.layer_type, self.units, self.dropout,
                                            return_sequences=False,
                                            return_state=True))

#Core Sequence to Sequence Model

In [8]:
class Seq2SeqModel():
    def __init__(self, embedding_dim, encoder_layers, decoder_layers, layer_type, units, dropout, attention=False):
        self.embedding_dim = embedding_dim
        self.encoder_layers = encoder_layers
        self.decoder_layers = decoder_layers
        self.layer_type = layer_type
        self.units = units
        self.dropout = dropout
        self.attention = attention
        self.stats = []
        self.batch_size = 128

    def build(self, loss, optimizer, metric):
        self.loss = loss
        self.optimizer = optimizer
        self.metric = metric

    def set_vocabulary(self, input_tokenizer, targ_tokenizer):
        self.input_tokenizer = input_tokenizer
        self.targ_tokenizer = targ_tokenizer
        self.create_model()
    
    def create_model(self):

        encoder_vocab_size = len(self.input_tokenizer.word_index) + 1
        decoder_vocab_size = len(self.targ_tokenizer.word_index) + 1

        self.encoder = Encoder(self.layer_type, self.encoder_layers, self.units, encoder_vocab_size,
                               self.embedding_dim, self.dropout)

        self.decoder = Decoder(self.layer_type, self.decoder_layers, self.units, decoder_vocab_size,
                               self.embedding_dim,  self.dropout, self.attention)

    @tf.function
    def train_step(self, input, target, enc_state):

        loss = 0 

        with tf.GradientTape() as tape: 

            enc_out, enc_state = self.encoder(input, enc_state)

            dec_state = enc_state
            dec_input = tf.expand_dims([self.targ_tokenizer.word_index["\t"]]*self.batch_size ,1)

            #Teacher Forcing

            if random.random() < self.teacher_forcing_ratio:

                for t in range(1, target.shape[1]):

                    preds, dec_state, _ = self.decoder(dec_input, dec_state, enc_out)
                    loss += self.loss(target[:,t], preds)
                    self.metric.update_state(target[:,t], preds)
                    
                    dec_input = tf.expand_dims(target[:,t], 1)
            
            else:

                for t in range(1, target.shape[1]):

                    preds, dec_state, _ = self.decoder(dec_input, dec_state, enc_out)
                    loss += self.loss(target[:,t], preds)
                    self.metric.update_state(target[:,t], preds)

                    preds = tf.argmax(preds, 1)
                    dec_input = tf.expand_dims(preds, 1)


            batch_loss = loss / target.shape[1]

            variables = self.encoder.variables + self.decoder.variables
            gradients = tape.gradient(loss, variables)

            self.optimizer.apply_gradients(zip(gradients, variables))

        return batch_loss, self.metric.result()

    @tf.function
    def validation_step(self, input, target, enc_state):

        loss = 0
        
        enc_out, enc_state = self.encoder(input, enc_state)

        dec_state = enc_state
        dec_input = tf.expand_dims([self.targ_tokenizer.word_index["\t"]]*self.batch_size ,1)

        for t in range(1, target.shape[1]):

            preds, dec_state, _ = self.decoder(dec_input, dec_state, enc_out)
            loss += self.loss(target[:,t], preds)
            self.metric.update_state(target[:,t], preds)

            preds = tf.argmax(preds, 1)
            dec_input = tf.expand_dims(preds, 1)

        batch_loss = loss / target.shape[1]
        
        return batch_loss, self.metric.result()


    def fit(self, dataset, val_dataset, batch_size=128, epochs=10, use_wandb=False, teacher_forcing_ratio=1.0):

        self.batch_size = batch_size
        self.teacher_forcing_ratio = teacher_forcing_ratio

        steps_per_epoch = len(dataset) // self.batch_size
        steps_per_epoch_val = len(val_dataset) // self.batch_size
        
        dataset = dataset.batch(self.batch_size, drop_remainder=True)
        val_dataset = val_dataset.batch(self.batch_size, drop_remainder=True)

        #Translation
        sample_inp, sample_targ = next(iter(dataset))
        self.max_target_len = sample_targ.shape[1]
        self.max_input_len = sample_inp.shape[1]

        template = "\nTrain Loss: {0:.4f} Train Accuracy: {1:.4f} Validation Loss: {2:.4f} Validation Accuracy: {3:.4f}"

        print("-"*100)
        for epoch in range(1, epochs+1):
            print(f"EPOCH {epoch}\n")

            
            total_loss = 0
            total_acc = 0
            self.metric.reset_states()

            starting_time = time.time()
            enc_state = self.encoder.initialize_hidden_state(self.batch_size)

            print("Training ...\n")
            for batch, (input, target) in enumerate(dataset.take(steps_per_epoch)):
                batch_loss, acc = self.train_step(input, target, enc_state)
                total_loss += batch_loss
                total_acc += acc


                if batch==0 or ((batch + 1) % 100 == 0):
                    print(f"Batch {batch+1} Loss {batch_loss:.4f}")

            avg_acc = total_acc / steps_per_epoch
            avg_loss = total_loss / steps_per_epoch

            # Validation
            total_val_loss = 0
            total_val_acc = 0
            self.metric.reset_states()

            enc_state = self.encoder.initialize_hidden_state(self.batch_size)

            print("\nValidating ...")
            for batch, (input, target) in enumerate(val_dataset.take(steps_per_epoch_val)):
                batch_loss, acc = self.validation_step(input, target, enc_state)
                total_val_loss += batch_loss
                total_val_acc += acc

            avg_val_acc = total_val_acc / steps_per_epoch_val
            avg_val_loss = total_val_loss / steps_per_epoch_val

            print(template.format(avg_loss, avg_acc*100, avg_val_loss, avg_val_acc*100))
            
            time_taken = time.time() - starting_time
            self.stats.append({"epoch": epoch,
                            "train loss": avg_loss,
                            "val loss": avg_val_loss,
                            "train acc": avg_acc*100,
                            "val acc": avg_val_acc*100,
                            "training time": time_taken})
            
            if use_wandb:
                wandb.log(self.stats[-1])
            
            print(f"\nTime taken for the epoch {time_taken:.4f}")
            print("-"*100)
        
        print("\nModel trained successfully !!")
        
    def evaluate(self, test_dataset, batch_size=None):

        if batch_size is not None:
            self.batch_size = batch_size

        steps_per_epoch_test = len(test_dataset) // batch_size
        test_dataset = test_dataset.batch(batch_size, drop_remainder=True)
        
        total_test_loss = 0
        total_test_acc = 0
        self.metric.reset_states()

        enc_state = self.encoder.initialize_hidden_state(self.batch_size)

        print("\nRunning test dataset through the model...\n")
        for batch, (input, target) in enumerate(test_dataset.take(steps_per_epoch_test)):
            batch_loss, acc = self.validation_step(input, target, enc_state)
            total_test_loss += batch_loss
            total_test_acc += acc

        avg_test_acc = total_test_acc / steps_per_epoch_test
        avg_test_loss = total_test_loss / steps_per_epoch_test
    
        print(f"Test Loss: {avg_test_loss:.4f} Test Accuracy: {avg_test_acc:.4f}")

        return avg_test_loss, avg_test_acc


    def translate(self, word, get_heatmap=False):

        word = "\t" + word + "\n"

        inputs = self.input_tokenizer.texts_to_sequences([word])
        inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                               maxlen=self.max_input_len,
                                                               padding="post")

        result = ""
        att_wts = []

        enc_state = self.encoder.initialize_hidden_state(1)
        enc_out, enc_state = self.encoder(inputs, enc_state)

        dec_state = enc_state
        dec_input = tf.expand_dims([self.targ_tokenizer.word_index["\t"]]*1, 1)

        for t in range(1, self.max_target_len):

            preds, dec_state, attention_weights = self.decoder(dec_input, dec_state, enc_out)
            
            if get_heatmap:
                att_wts.append(attention_weights)
            
            preds = tf.argmax(preds, 1)
            next_char = self.targ_tokenizer.index_word[preds.numpy().item()]
            result += next_char

            dec_input = tf.expand_dims(preds, 1)

            if next_char == "\n":
                return result[:-1], att_wts[:-1]

        return result[:-1], att_wts[:-1]


In [9]:
def randomly_evaluate(model, test_file=get_data_files("hi")[2], n=10):

    df = pd.read_csv(test_file, sep="\t", header=None)
    df = df.sample(n=n).reset_index(drop=True)

    print(f"Randomly evaluating the model on {n} words\n")

    for i in range(n):
        word = str(df[1][i])

        print(f"Input word: {word}")
        print(f"Actual translation: {str(df[0][i])}")
        print(f"Model translation: {model.translate(word)[0]}\n")


def test_on_dataset(language, embedding_dim, encoder_layers, decoder_layers, layer_type, units, dropout, attention, teacher_forcing_ratio=1.0, save_outputs=None):
    
    TRAIN_TSV, VAL_TSV, TEST_TSV = get_data_files(language)

    model = Seq2SeqModel(embedding_dim, 
                         encoder_layers, 
                         decoder_layers, 
                         layer_type, 
                         units,
                         dropout,
                         attention)

    dataset, input_tokenizer, targ_tokenizer = preprocess_data(TRAIN_TSV)
    val_dataset, _, _ = preprocess_data(VAL_TSV, input_tokenizer, targ_tokenizer)

    model.set_vocabulary(input_tokenizer, targ_tokenizer)
    model.build(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                optimizer = tf.keras.optimizers.Adam(),
                metric = tf.keras.metrics.SparseCategoricalAccuracy())
    
    model.fit(dataset, val_dataset, epochs=20, use_wandb=False, teacher_forcing_ratio=teacher_forcing_ratio)


    test_dataset, _, _ = preprocess_data(TEST_TSV, model.input_tokenizer, model.targ_tokenizer)
    test_loss, test_acc = model.evaluate(test_dataset, batch_size=100)

    ##  Wordaccuracy
    test_tsv = pd.read_csv(TEST_TSV, sep="\t", header=None)
    inputs = test_tsv[1].astype(str).tolist()
    targets = test_tsv[0].astype(str).tolist()
    
    outputs = []

    for word in inputs:
        outputs.append(model.translate(word)[0])

    def word_level_acc(outputs, targets):
        return np.sum(np.asarray(outputs) == np.array(targets)) / len(outputs)

    print(f"Word level accuracy: {word_level_acc(outputs, targets)}")

    if save_outputs is not None:
        df = pd.DataFrame()
        df["inputs"] = inputs
        df["targets"] = targets
        df["outputs"] = outputs
        df.to_csv(save_outputs)


    return model


#Model Connectivity Visualization

In [10]:
def get_lstm_output(decoder, x, hidden, enc_out=None):
    
    x = decoder.embedding_layer(x)

    if decoder.attention:
        context_vector, attention_weights = decoder.attention_layer(hidden, enc_out)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], -1)
    else:
        attention_weights = None

    x = decoder.rnn_layers[0](x, initial_state=hidden)

    for layer in decoder.rnn_layers[1:]:
        x = layer(x)

    output, state = x[0], x[1:]

    
    return output, state, attention_weights

def get_output_from_embedding(encoder, x, hidden):

    x = encoder.rnn_layers[0](x, initial_state=hidden)

    for layer in encoder.rnn_layers[1:]:
        x = layer(x)

    output, state = x[0], x[1:]

    return output, state


def get_connectivity(model, word):

    word = "\t" + word + "\n"

    inputs = model.input_tokenizer.texts_to_sequences([word])
    inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs,
                                                            maxlen=model.max_input_len,
                                                            padding="post")

    result = ""

    gradient_list = []

    enc_state = model.encoder.initialize_hidden_state(1)
    embedded_in = model.encoder.embedding(inputs)


    with tf.GradientTape(persistent=True, watch_accessed_variables=False) as tape:
        tape.watch(embedded_in)

        enc_out, enc_state = get_output_from_embedding(model.encoder, embedded_in, enc_state)

        dec_state = enc_state
        dec_input = tf.expand_dims([model.targ_tokenizer.word_index["\t"]]*1, 1)

        for t in range(1, model.max_target_len):

            lstm_out, dec_state, _ = get_lstm_output(model.decoder, dec_input, dec_state, enc_out)

            preds = model.decoder.dense(model.decoder.flatten(lstm_out))
            gradient_list.append(tape.gradient(lstm_out, embedded_in)[0])
            
            preds = tf.argmax(preds, 1)
            next_char = model.targ_tokenizer.index_word[preds.numpy().item()]
            result += next_char

            dec_input = tf.expand_dims(preds, 1)

            if next_char == "\n":
                return result[:-1], gradient_list[:-1]

        return result[:-1], gradient_list[:-1]

In [11]:
from sklearn.preprocessing import MinMaxScaler
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

from IPython.display import HTML as html_print
from IPython.display import display
import tensorflow.keras.backend as K

#html 
def cstr(s, color='black'):
    if s == ' ':
      return "<text style=color:#000;padding-left:10px;background-color:{}> </text>".format(color, s)
    else:
      return "<text style=color:#000;background-color:{}>{} </text>".format(color, s)
	
# print html
def print_color(t):
	  display(html_print(''.join([cstr(ti, color=ci) for ti,ci in t])))

# get color
def get_clr(value):
    colors = ['#85c2e1', '#89c4e2', '#95cae5', '#99cce6', '#a1d0e8'
      '#b2d9ec', '#baddee', '#c2e1f0', '#eff7fb', '#f9e8e8',
      '#f9e8e8', '#f9d4d4', '#f9bdbd', '#f8a8a8', '#f68f8f',
      '#f47676', '#f45f5f', '#f34343', '#f33b3b', '#f42e2e']
    value = int(value * 19)
    if value == 19:
        value -= 1
    return colors[value]

def sigmoid(x):
    z = 1/(1 + np.exp(-x)) 
    return z

def softmax(x):
    v = np.exp(x)
    v = v / np.sum(v)
    return v

def get_gradient_norms(grad_list, word, activation="sigmoid"):
    grad_norms = []
    for grad_tensor in grad_list:
        grad_mags = tf.norm(grad_tensor, axis=1)
        grad_mags = grad_mags[:len(word)]
        if activation == "softmax":
            grad_mags_scaled = softmax(grad_mags)
        elif activation == "scaler":
            scaler = MinMaxScaler()
            grad_mags = tf.reshape(grad_mags, (-1,1))
            grad_mags_scaled = scaler.fit_transform(grad_mags)
        else:
            grad_mags_scaled = sigmoid(grad_mags)
        grad_norms.append(grad_mags_scaled)
    return grad_norms

def visualize(grad_norms, word, translated_word):
    print("Original Word:", word)
    print("Transliterated Word:", translated_word)
    for i in range(len(translated_word)):
        print("Connectivity Visualization for", translated_word[i],":")
        text_colours = []
        for j in range(len(grad_norms[i])):
            text = (word[j], get_clr(grad_norms[i][j]))
            text_colours.append(text)
        print_color(text_colours)

def visualise_connectivity(model, word, activation="sigmoid"):
    translated_word, grad_list = get_connectivity(model, word)
    grad_norms = get_gradient_norms(grad_list, word, activation)
    visualize(grad_norms, word, translated_word)

#Testing without attention

In [13]:
#Testing on the dataset with the best hyper parameters found after sweeps.
model = test_on_dataset(language="hi",
                        embedding_dim=256,
                        encoder_layers=3,
                        decoder_layers=3,
                        layer_type="lstm",
                        units=256,
                        dropout=0.2,
                        attention=False, save_outputs='predictions_vanilla.csv')

----------------------------------------------------------------------------------------------------
EPOCH 1

Training ...

Batch 1 Loss 3.9883
Batch 100 Loss 1.0728
Batch 200 Loss 0.9839
Batch 300 Loss 0.9155

Validating ...

Train Loss: 1.1216 Train Accuracy: 66.9055 Validation Loss: 2.8826 Validation Accuracy: 44.6775

Time taken for the epoch 120.6969
----------------------------------------------------------------------------------------------------
EPOCH 2

Training ...

Batch 1 Loss 0.8923
Batch 100 Loss 0.8731
Batch 200 Loss 0.7926
Batch 300 Loss 0.7589

Validating ...

Train Loss: 0.8292 Train Accuracy: 74.8300 Validation Loss: 3.2128 Validation Accuracy: 45.6532

Time taken for the epoch 33.6026
----------------------------------------------------------------------------------------------------
EPOCH 3

Training ...

Batch 1 Loss 0.7498
Batch 100 Loss 0.6860
Batch 200 Loss 0.5596
Batch 300 Loss 0.5896

Validating ...

Train Loss: 0.6396 Train Accuracy: 79.4184 Validation Loss

#Testing with attention

In [16]:
model2 = test_on_dataset(language="hi",
                        embedding_dim=256,
                        encoder_layers=3,
                        decoder_layers=3,
                        layer_type="lstm",
                        units=256,
                        dropout=0.2,
                        attention=True, save_outputs='predictions_attention.csv')

----------------------------------------------------------------------------------------------------
EPOCH 1

Training ...

Batch 1 Loss 3.9899
Batch 100 Loss 1.0848
Batch 200 Loss 0.9344
Batch 300 Loss 0.8751

Validating ...

Train Loss: 1.0525 Train Accuracy: 68.1665 Validation Loss: 2.2913 Validation Accuracy: 52.4259

Time taken for the epoch 136.9153
----------------------------------------------------------------------------------------------------
EPOCH 2

Training ...

Batch 1 Loss 0.8645
Batch 100 Loss 0.8185
Batch 200 Loss 0.7586
Batch 300 Loss 0.6920

Validating ...

Train Loss: 0.7778 Train Accuracy: 75.8738 Validation Loss: 2.4905 Validation Accuracy: 52.4783

Time taken for the epoch 40.7995
----------------------------------------------------------------------------------------------------
EPOCH 3

Training ...

Batch 1 Loss 0.6454
Batch 100 Loss 0.6030
Batch 200 Loss 0.5231
Batch 300 Loss 0.4283

Validating ...

Train Loss: 0.5481 Train Accuracy: 80.8016 Validation Loss

In [17]:
def get_test_words(n):
    test_df = pd.read_csv(get_data_files("hi")[2])
    test_sample = test_df.sample(n)
    test_sample.reset_index(inplace=True, drop=True)
    test_words = []
    for i in test_sample.index:
        entry = test_sample["अंक\tank\t5"].loc[i]
        parts = entry.split("\t")
        word = parts[1]
        test_words.append(word)
    return test_words

test_words = get_test_words(5)
print(test_words)

['teekamgarh', 'sandli', 'singer', 'brahmleen', 'sandrbh']


In [18]:
for word in test_words:
    visualise_connectivity(model2, word, activation="scaler")

Original Word: teekamgarh
Transliterated Word: तीकामगढ़
Connectivity Visualization for त :


Connectivity Visualization for ी :


Connectivity Visualization for क :


Connectivity Visualization for ा :


Connectivity Visualization for म :


...

Original Word: sandrbh
Transliterated Word: संदृभ
Connectivity Visualization for स :


Connectivity Visualization for ं :


Connectivity Visualization for द :


Connectivity Visualization for ृ :


Connectivity Visualization for भ :


In [14]:
randomly_evaluate(model, n=5)

Randomly evaluating the model on 5 words

Input word: nihaalchand
Actual translation: निहालचंद
Model translation: निहीलचंद

Input word: paheliyan
Actual translation: पहेलियां
Model translation: पहिलियां

Input word: then
Actual translation: थेन
Model translation: थें

Input word: batlaakar
Actual translation: बतलाकर
Model translation: बतलाकर

Input word: nabj
Actual translation: नब्ज़
Model translation: नबाज



In [16]:
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

#WandB training

In [17]:
def train_with_wandb(language):

    config_defaults = {"embedding_dim": 64, 
                       "enc_dec_layers": 1,
                       "layer_type": "lstm",
                       "units": 128,
                       "dropout": 0,
                       "attention": False,
                       "teacher_forcing_ratio": 1.0
                       }

    wandb.init(config=config_defaults, project="cs6910-assignment3", resume=True)
    # Below is an example of a custom run name for sweep 4
    # This line was different for all sweeps

    ## 1. SELECT LANGUAGE ##
    TRAIN_TSV, VAL_TSV, TEST_TSV = get_data_files(language)

    ## 2. DATA PREPROCESSING ##
    dataset, input_tokenizer, targ_tokenizer = preprocess_data(TRAIN_TSV)
    val_dataset, _, _ = preprocess_data(VAL_TSV, input_tokenizer, targ_tokenizer)

    ## 3. CREATING THE MODEL ##
    model = Seq2SeqModel(embedding_dim=wandb.config.embedding_dim,
                         encoder_layers=wandb.config.enc_dec_layers,
                         decoder_layers=wandb.config.enc_dec_layers,
                         layer_type=wandb.config.layer_type,
                         units=wandb.config.units,
                         dropout=wandb.config.dropout,
                         attention=wandb.config.attention)
    
    ## 4. COMPILING THE MODEL 
    model.set_vocabulary(input_tokenizer, targ_tokenizer)
    model.build(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                optimizer = tf.keras.optimizers.Adam(),
                metric = tf.keras.metrics.SparseCategoricalAccuracy())
    
    ## 5. FITTING AND VALIDATING THE MODEL
    model.fit(dataset, val_dataset, epochs=20, use_wandb=True, teacher_forcing_ratio=wandb.config.teacher_forcing_ratio)

#First sweep

In [20]:
sweep_config = {
  "name": "Sweep 1- Assignment3",
  "method": "grid",
  "parameters": {
        "enc_dec_layers": {
           "values": [1, 2, 3]
        },
        "units": {
            "values": [64, 128, 256]
        },
        "layer_type": {
            "values": ["rnn", "gru", "lstm"]
        }
    }
}

In [25]:
sweep_id = wandb.sweep(sweep_config, project="cs6910-assignment3")

Create sweep with ID: l64qp4dv
Sweep URL: https://wandb.ai/harshagudivada/cs6910-assignment3/sweeps/l64qp4dv


In [26]:
wandb.agent(sweep_id, function=lambda: train_with_wandb("hi"))

wandb: Agent Starting Run: s15251gu with config:
wandb: 	enc_dec_layers: 1
wandb: 	layer_type: rnn
wandb: 	units: 64
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Tried to auto resume run with id wqwfmy1i but id s15251gu is set.


----------------------------------------------------------------------------------------------------
EPOCH 1

Training ...

Batch 1 Loss 4.0162
Batch 100 Loss 1.1844
Batch 200 Loss 1.0742
Batch 300 Loss 0.9797

Validating ...

Train Loss: 1.2067 Train Accuracy: 63.2089 Validation Loss: 2.3059 Validation Accuracy: 47.9538

Time taken for the epoch 27.4635
----------------------------------------------------------------------------------------------------
EPOCH 2

Training ...

Batch 1 Loss 0.9280
Batch 100 Loss 0.9105
Batch 200 Loss 0.8629
Batch 300 Loss 0.8310

Validating ...

Train Loss: 0.8789 Train Accuracy: 74.9216 Validation Loss: 2.6213 Validation Accuracy: 48.3824

Time taken for the epoch 12.8931
----------------------------------------------------------------------------------------------------
EPOCH 3

Training ...

Batch 1 Loss 0.8129
Batch 100 Loss 0.8283
Batch 200 Loss 0.7775
Batch 300 Loss 0.7671

Validating ...

Train Loss: 0.8017 Train Accuracy: 77.0058 Validation Loss:

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train acc,▁▅▆▆▇▇▇▇▇▇▇█████████
train loss,█▅▄▃▃▃▂▂▂▂▂▂▂▁▁▁▁▁▁▁
training time,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val acc,▇▇▇█▃▃▂▁▃▃▅▃▂▂▃▃▂▄▂▃
val loss,▁▂▃▃▅▅▆▆▆▆▆▆▇▇▇▇█▇██
epoch,20
train acc,83.92747
train loss,0.51406
training time,13.00186
val acc,44.06693


wandb: Agent Starting Run: 2bxog4mc with config:
wandb: 	enc_dec_layers: 1
wandb: 	layer_type: rnn
wandb: 	units: 128
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


----------------------------------------------------------------------------------------------------
EPOCH 1

Training ...

Batch 1 Loss 4.0132
Batch 100 Loss 1.0891
Batch 200 Loss 1.0175
Batch 300 Loss 0.9905

Validating ...

Train Loss: 1.1449 Train Accuracy: 65.9362 Validation Loss: 3.4433 Validation Accuracy: 36.5478

Time taken for the epoch 29.1032
----------------------------------------------------------------------------------------------------
EPOCH 2

Training ...

Batch 1 Loss 0.9687
Batch 100 Loss 0.9336
Batch 200 Loss 0.9290
Batch 300 Loss 0.9605

Validating ...

Train Loss: 0.9431 Train Accuracy: 73.0482 Validation Loss: 3.5167 Validation Accuracy: 36.1969

Time taken for the epoch 14.8346
----------------------------------------------------------------------------------------------------
EPOCH 3

Training ...

Batch 1 Loss 0.9328
Batch 100 Loss 0.8830
Batch 200 Loss 0.9686
Batch 300 Loss 0.9186

Validating ...

Train Loss: 0.9145 Train Accuracy: 73.4536 Validation Loss:

...

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train acc,▁▃▃▄▄▅▅▆▆▆▇▇▇▇██████
train loss,█▆▅▅▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁▁
training time,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val acc,▁▁▁▃▃▃▄▄▅▅▆▆▆▇▇▇▇▇██
val loss,▄▆█▅▆▆▅▅▄▄▃▃▃▂▂▂▂▂▁▁
epoch,20
train acc,95.09074
train loss,0.15074
training time,28.93486
val acc,77.23409


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: v4epzckc with config:
wandb: 	enc_dec_layers: 3
wandb: 	layer_type: lstm
wandb: 	units: 256
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


----------------------------------------------------------------------------------------------------
EPOCH 1

Training ...

Batch 1 Loss 3.9905
Batch 100 Loss 1.1456
Batch 200 Loss 1.0155
Batch 300 Loss 0.9185

Validating ...

Train Loss: 1.1417 Train Accuracy: 66.2528 Validation Loss: 2.6774 Validation Accuracy: 47.4621

Time taken for the epoch 118.1029
----------------------------------------------------------------------------------------------------
EPOCH 2

Training ...

Batch 1 Loss 0.9485
Batch 100 Loss 0.9184
Batch 200 Loss 0.8157
Batch 300 Loss 0.7488

Validating ...

Train Loss: 0.8504 Train Accuracy: 74.2592 Validation Loss: 3.4207 Validation Accuracy: 42.8704

Time taken for the epoch 29.4493
----------------------------------------------------------------------------------------------------
EPOCH 3

Training ...

Batch 1 Loss 0.8342
Batch 100 Loss 0.8083
Batch 200 Loss 0.6953
Batch 300 Loss 0.6319

Validating ...

Train Loss: 0.7101 Train Accuracy: 77.1758 Validation Loss

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train acc,▁▃▃▄▅▆▆▇▇▇▇▇████████
train loss,█▆▅▄▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁
training time,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val acc,▂▁▃▃▄▅▆▆▆▇▇█████████
val loss,▅█▅▅▄▄▃▃▂▁▁▁▁▁▁▂▁▂▂▂
epoch,20
train acc,98.34455
train loss,0.05104
training time,29.56824
val acc,80.36323


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


#Second sweep

In [18]:
sweep_config2 = {
  "name": "Sweep 2- Assignment3",
  "method": "grid",
  "parameters": {
        "enc_dec_layers": {
           "values": [2, 3]
        },
        "embedding_dim": {
            "values": [64, 128, 256]
        },
        "dropout": {
            "values": [0.2, 0.3]
        }
    }
}

In [19]:
sweep_id2 = wandb.sweep(sweep_config2, project="cs6910-assignment3")

Create sweep with ID: p25mgmof
Sweep URL: https://wandb.ai/harshagudivada/cs6910-assignment3/sweeps/p25mgmof


In [20]:
wandb.agent(sweep_id2, function=lambda: train_with_wandb("hi"))

wandb: Agent Starting Run: 7hnygblg with config:
wandb: 	dropout: 0.2
wandb: 	embedding_dim: 64
wandb: 	enc_dec_layers: 2
wandb: Currently logged in as: harshagudivada. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


----------------------------------------------------------------------------------------------------
EPOCH 1

Training ...

Batch 1 Loss 3.9933
Batch 100 Loss 1.1702
Batch 200 Loss 1.1788
Batch 300 Loss 0.9235

Validating ...

Train Loss: 1.1972 Train Accuracy: 65.9775 Validation Loss: 2.3918 Validation Accuracy: 48.2350

Time taken for the epoch 200.6497
----------------------------------------------------------------------------------------------------
EPOCH 2

Training ...

Batch 1 Loss 0.9465
Batch 100 Loss 0.9016
Batch 200 Loss 0.8876
Batch 300 Loss 0.8749

Validating ...

Train Loss: 0.9144 Train Accuracy: 73.3141 Validation Loss: 2.2572 Validation Accuracy: 51.4902

Time taken for the epoch 115.5331
----------------------------------------------------------------------------------------------------
EPOCH 3

Training ...

Batch 1 Loss 0.8992
Batch 100 Loss 0.8472
Batch 200 Loss 0.8525
Batch 300 Loss 0.8062

Validating ...

Train Loss: 0.8476 Train Accuracy: 74.7927 Validation Los

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train acc,▁▃▃▃▄▄▅▅▆▆▇▇▇▇▇█████
train loss,█▆▆▅▅▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁
training time,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val acc,▁▂▁▂▂▂▃▃▄▄▄▅▆▆▇▇▇███
val loss,▆▅█▇▇▇▇▇▅▆▅▅▃▄▃▂▂▁▂▁
epoch,20
train acc,94.62084
train loss,0.16484
training time,114.22137
val acc,76.64292


wandb: Agent Starting Run: zvirp5si with config:
wandb: 	dropout: 0.2
wandb: 	embedding_dim: 64
wandb: 	enc_dec_layers: 3
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


----------------------------------------------------------------------------------------------------
EPOCH 1

Training ...

Batch 1 Loss 3.9898
Batch 100 Loss 1.2218
Batch 200 Loss 1.1181
Batch 300 Loss 1.0975

Validating ...

Train Loss: 1.2713 Train Accuracy: 64.4884 Validation Loss: 2.5254 Validation Accuracy: 42.7320

Time taken for the epoch 298.4471
----------------------------------------------------------------------------------------------------
EPOCH 2

Training ...

Batch 1 Loss 1.0666
Batch 100 Loss 1.0023
Batch 200 Loss 0.9493
Batch 300 Loss 0.8662

Validating ...

Train Loss: 0.9667 Train Accuracy: 72.7097 Validation Loss: 3.1002 Validation Accuracy: 41.8566

Time taken for the epoch 177.4797
----------------------------------------------------------------------------------------------------
EPOCH 3

Training ...

Batch 1 Loss 0.9235
Batch 100 Loss 0.8931
Batch 200 Loss 0.9026
Batch 300 Loss 0.8374

Validating ...

Train Loss: 0.8936 Train Accuracy: 73.8756 Validation Los

...

----------------------------------------------------------------------------------------------------
EPOCH 1

Training ...

Batch 1 Loss 3.9892
Batch 100 Loss 1.1997
Batch 200 Loss 1.0520
Batch 300 Loss 1.0204

Validating ...

Train Loss: 1.2087 Train Accuracy: 65.8216 Validation Loss: 3.0124 Validation Accuracy: 38.2768

Time taken for the epoch 305.0879
----------------------------------------------------------------------------------------------------
EPOCH 2

Training ...

Batch 1 Loss 0.9836
Batch 100 Loss 0.9459
Batch 200 Loss 0.8918
Batch 300 Loss 0.8631

Validating ...

Train Loss: 0.9097 Train Accuracy: 73.3048 Validation Loss: 2.8802 Validation Accuracy: 44.8117

Time taken for the epoch 181.2504
----------------------------------------------------------------------------------------------------
EPOCH 3

Training ...

Batch 1 Loss 0.8906
Batch 100 Loss 0.8441
Batch 200 Loss 0.7812
Batch 300 Loss 0.7941

Validating ...

Train Loss: 0.8260 Train Accuracy: 75.1906 Validation Los

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
train acc,▁▃▃▄▄▅▅▆▆▇▇▇▇▇██████
train loss,█▆▅▅▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁
training time,█▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁
val acc,▁▂▃▃▃▄▄▅▅▅▆▆▆▆▇▇▇▇██
val loss,█▇▆▇█▆▆▄▆▅▅▄▄▄▂▃▂▂▂▁
epoch,20
train acc,95.18855
train loss,0.14661
training time,182.89684
val acc,77.23711


wandb: Agent Starting Run: 8nezn39m with config:
wandb: 	dropout: 0.3
wandb: 	embedding_dim: 128
wandb: 	enc_dec_layers: 2
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


----------------------------------------------------------------------------------------------------
EPOCH 1

Training ...



wandb: Ctrl + C detected. Stopping sweep.
